In [463]:
import pandas as pd
#from pandas import ExcelWriter
#from pandas import ExcelFile
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn import decomposition
from matplotlib.mlab import PCA
from math import sin, cos, sqrt, atan2, radians

def read_data_vt_mt():
 print('Reading the data ...')
 VT = pd.read_csv('VT_cleaned.csv',low_memory=False)
 MT = pd.read_csv('MT_cleaned.csv',low_memory=False)
 return (VT, MT)



In [464]:

vt, mt   = read_data_vt_mt()
print('Vermont', vt.shape)
print('Montana', mt.shape)

Reading the data ...
Vermont (283285, 23)
Montana (825118, 33)


In [465]:
temp=mt.groupby(['driver_gender']).agg(['count'])
n_females_mt=temp.iloc[0,1];
n_males_mt=temp.iloc[1,1]
n_tot_mt=n_males_mt+n_females_mt

temp=vt.groupby(['driver_gender']).agg(['count'])
n_females_vt=temp.iloc[0,1];
n_males_vt=temp.iloc[1,1]
n_tot_vt=n_males_vt+n_females_vt


print('MT male stop ratio: %.11f' %(n_males_mt/(n_tot_mt)))

MT male stop ratio: 0.67507233342


In [466]:
temp = mt.groupby(['out_of_state','is_arrested' ]).agg(['count'])

In [467]:
in_state_arrest_mt=temp.iloc[1,3]
out_state_arrest_mt=temp.iloc[3,3]
arrest_out_in_proportion=out_state_arrest_mt/in_state_arrest_mt
print('How many more times likely are you to be arrested in MT if out of state:')
print(arrest_out_in_proportion)

How many more times likely are you to be arrested in MT if out of state:
0.399294445812


In [468]:
temp=mt.groupby(['is_arrested']).agg(['count'])
not_ar_mt=temp.iloc[0,0]
ar_mt=temp.iloc[1,0]
ar_pro_mt=ar_mt/(ar_mt+not_ar_mt)
temp=vt.groupby(['is_arrested']).agg(['count'])
not_ar_vt=temp.iloc[0,0]
ar_vt=temp.iloc[1,0]
ar_pro_vt=ar_vt/(ar_vt+not_ar_vt)
chai_s_res=((ar_pro_vt-ar_pro_mt)**2)/min(ar_pro_vt,ar_pro_mt)
print('chai square test result')
print(chai_s_res)



chai square test result
0.00701315041071


In [469]:
temp = mt.groupby(['violation']).agg(['count'])

In [471]:
count = temp.loc[temp.index.str.contains('Speeding'), temp.columns[0]].sum()
print('Speeding proportion in Montana')
print(count/n_tot_mt)

Speeding proportion in Montana


0.65819473720574206

In [472]:
dui_pro_mt=temp.loc[temp.index.str.contains('DUI'), temp.columns[0]].sum()/n_tot_mt


In [473]:
temp = vt.groupby(['violation']).agg(['count'])
dui_pro_vt=temp.loc[temp.index.str.contains('DUI'), temp.columns[0]].sum()/n_tot_vt

print('How much more likely does a traffic stop in Montana result in a DUI than a traffic stop in Vermont?')
print(dui_pro_mt-dui_pro_vt)

How much more likely does a traffic stop in Montana result in a DUI than a traffic stop in Vermont?
0.00814480539729


In [478]:
mt['vehicle_year'] = mt['vehicle_year'].replace('UNK', 0)
mt['vehicle_year'] = mt['vehicle_year'].replace('NON-',0 )
mt['vehicle_year'] = pd.to_numeric(mt['vehicle_year'])
mt['stop_date'] = pd.to_datetime(mt['stop_date'])
mt['stop_year'] = mt['stop_date'].dt.year
mt = mt.fillna(0)

years=np.array([2010, 2011, 2012,2013,2014,2015])
av=np.array([0,0,0,0,0,0])


temp = mt.groupby(['stop_year'], as_index=False)['vehicle_year'].mean()

train_x = temp['stop_year'][1:]
train_y = temp['vehicle_year'][1:]

from scipy import stats

slope, intercept, r_value, p_value, std_err = stats.linregress(train_x,train_y)
print('slope:',slope)
print('intercept',intercept)
print('p value',p_value)
pred=2020*slope+intercept
print('Prediction in 2020:', pred)

#from sklearn import linear_model
#myclf = linear_model.Lasso(tol=1)
#myclf.fit([0,1],[2,3])
#print(clf.coef_)


slope: 0.976041449893
intercept 21.0266849183
p value 0.0535610298484
Prediction in 2020: 1992.6304137


In [481]:
mtsh = pd.to_datetime(mt['stop_time'])
vtsh = pd.to_datetime(vt['stop_time'])
vq = pd.concat([mtsh.dt.hour, vtsh.dt.hour])
rs = vq.groupby(vq)
vq_max=rs.agg('count').max()
print('number in max hour:',vq_max)
vq_min=rs.agg('count').min()
print('number in min hour:',vq_min)
print('difference between numbers in min/max hours:',vq_max-vq_min)

number in max hour: 95891
number in min hour: 547
difference between numbers in min/max hours: 95344


In [462]:
mt = mt[(mt.lon >(-117.004501) ) & (mt.lon<(-102.9445699))&(mt.lat >43.64999 ) & (mt.lat<49.0456001)]
varmt = mt.groupby(['county_name'], as_index=False)['lat', 'lon'].var()
meanmt = mt.groupby(['county_name'], as_index=False)['lat', 'lon'].mean()
R_Earth = 3959 *   1.60934
dlon = (((meanmt['lon'])) -  np.sqrt((varmt['lon'])))-( ((meanmt['lon'])) +  np.sqrt((varmt['lon'])))
dlat = (((meanmt['lat'])) - np.sqrt((varmt['lat'])))-(((meanmt['lat']))+ np.sqrt((varmt['lat'])))
qop=(np.sin(np.deg2rad(dlat) / 2))
con =1e-6*(qop**2+0.0001 * qop**4)+ 2 * np.arctan2(np.sqrt((qop**2+0.0001 * qop**4)), np.sqrt(1 - (qop**2+0.0001 * qop**4)))
temp = np.cos(np.deg2rad(((meanmt['lat'])))) * np.cos(np.deg2rad(((meanmt['lat'])))) * (np.sin(np.deg2rad(dlon) / 2))**2 + 2e-10*np.cos(np.deg2rad(((meanmt['lat']))))
C_Ar = np.pi * (R_Earth  * con) *  (R_Earth  *  2 * np.arctan2(np.sqrt(temp), np.sqrt(1 - temp)))
print('Largest county area:', C_Ar[C_Ar[1:].argmax()])


Largest county area: 8781.03423571


In [445]:
1e-6

1e-06